In [76]:
from deepface import DeepFace
import face_recognition

In [77]:
import torch
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from numpy import asarray
from PIL import Image
import torch.nn.functional as F
from emotion_detection.fer_data_utils import SkResize, HistEq, AddChannel, ToRGB
from vision_utils.custom_architectures import SepConvModel
#extra added
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model, PretrainedMT
from emotion_detection.evaluate import predict_fer as eval_man
from vision_utils.custom_architectures import PretrainedMT
from multitask_rag.evaluate import predict_utk as test_image

In [78]:
PATHRESNETAGR='/home/manish/Documents/democlassi/other/resnet_model_21_val_loss=4.275671.pth'
PATHSEPCONVAGR='/home/manish/Documents/democlassi/other/sep_conv_adam_model_33_val_loss=4.714899.pth'
PATHVGGAGR='/home/manish/Documents/democlassi/other/vgg_model_21_val_loss=4.139335.pth'

PATHIMAGE='/home/manish/Documents/fotos/pic2.jpg'
#PATHIMAGE='/home/manish/Documents/fotos/two-people.jpg'
PATHSEPCONV='/home/manish//Documents/democlassi/emotion-detection/sepconv_model_55_val_loss=1.175765.pth'

In [80]:
#emotion
model = SepConvModel()
model.load_state_dict(torch.load(PATHSEPCONV, map_location="cpu"))

#age-race-gender
resnet_model_agr = PretrainedMT(model_name='resnet')
resnet_model_agr.load_state_dict(torch.load(PATHRESNETAGR, map_location="cpu"))


#age-race-gender
sep_conv_model_agr = SepConvModelMT()
sep_conv_model_agr.load_state_dict(torch.load(PATHSEPCONVAGR, map_location="cpu")) 

#age-race-gender
vgg_model_agr = PretrainedMT(model_name='vgg')
vgg_model_agr.load_state_dict(torch.load(PATHVGGAGR, map_location="cpu")) 



<All keys matched successfully>

In [69]:
def preprocess_fer(image, transf_learn):
    if transf_learn:
        transf = transforms.Compose([
            transforms.ToTensor()
        ])
    else:
        transf = transforms.Compose([
            HistEq(),
            AddChannel(),
            transforms.ToTensor()
        ])
        
        
        print(transf(image))  

    return transf(image).to(torch.float32).unsqueeze_(0)

In [70]:
def predict_fer(image, model, transf_learn=True):

    # process image
    image = preprocess_fer(image, transf_learn)
    
    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    emotion = F.softmax(model(image), dim=1).detach().to('cpu').numpy()[0]
    target_names = ['Angry', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Surprised', 'Neutral']
    pred_label = target_names[np.argmax(emotion)]

    emotion_probs = dict(zip(target_names, emotion))

    return emotion_probs, pred_label

In [73]:
def democlassiEvaluate(pil_image,):
    
    img_tensor= data_transforms(pil_image) 
    imge = pil_image.convert('L')
    img = imge.resize((48,48),Image.Resampling.LANCZOS)
    numpydata = asarray(img)
    res1=predict_fer(numpydata,model,True)
    # with resent model--age--race--gender
    #res2=test_image(img_tensor,resnet_model)
    #with sep_conv_model --age--gender---race
    #res2=test_image(img_tensor,sep_conv_model)
    #with VGG model ---age--race--gender
    res2=test_image(img_tensor,vgg_model_agr)
    
    #for resnet model
    res2 = test_image(img_tensor,resnet_model_agr)
    
    #for sepconv_ model
    res2 = test_image(img_tensor,sepconv_model_agr)
    return res1,res2 

In [74]:
def printDemoClassiOutput(res1,res2):
    print("Democlassi comparision")
    print("---Democlassi---")
    print("Age:---",{round(res2[0], 0)},"---")
    print("Gender:---",{res2[2]},"---")
    print("Race:---",{res2[4]},"---")
    print("Emotion:---",{res1[1]},"---") #res1 for emotion

In [75]:
import os
from os import listdir

folder_dir = "/home/manish/Documents/fotos"
save_dir = "/home/manish/Documents/fotos/after-cropping"

for images in os.listdir(folder_dir):
    if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            new_dir=save_dir+"/"+"presave.jpg"
            image = Image.open(new)

            tmp= image.size
            if( tmp[0]*tmp[1] > 400000 ):  #or tmp[0]*tmp[1] < 50625):
                image =image.resize((600,600),Image.Resampling.LANCZOS)
                
            if( tmp[0]*tmp[1] < 50625):  #or tmp[0]*tmp[1] < 50625):
                image =image.resize((225,225),Image.Resampling.LANCZOS)
                
            #image = image.convert("RGB")
            image.save(new_dir,"JPEG")
            
            image = face_recognition.load_image_file(new)
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
           
            for face_location in face_locations:
                # Print the location of each face in this image
                top, right, bottom, left = face_location
                face_image = image[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                
                #democlassi
                res1, res2 = democlassiEvaluate(pil_image)
                printDemoClassiOutput(res1,res2)
                print( '------------------------------------------- ')
    print('++++++++++++++++++++++++++++++++++++++++++')
print(res1)
print(res2)

/home/manish/Documents/fotos/pic3.jpg
Democlassi comparision
---Democlassi---
Age:--- {24.0} ---
Gender:--- {'Man'} ---
Race:--- {'Black'} ---
Emotion:--- {'Neutral'} ---
------------------------------------------- 
++++++++++++++++++++++++++++++++++++++++++
/home/manish/Documents/fotos/pic8.jpg
Democlassi comparision
---Democlassi---
Age:--- {29.0} ---
Gender:--- {'Woman'} ---
Race:--- {'White'} ---
Emotion:--- {'Happy'} ---
------------------------------------------- 
++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++
/home/manish/Documents/fotos/pic0.jpg
Democlassi comparision
---Democlassi---
Age:--- {23.0} ---
Gender:--- {'Man'} ---
Race:--- {'White'} ---
Emotion:--- {'Happy'} ---
------------------------------------------- 
++++++++++++++++++++++++++++++++++++++++++
/home/manish/Documents/fotos/pic12.jpeg
Democlassi comparision
---Democlassi---
Age:--- {28.0} ---
Gender:--- {'Woman'} ---
Race:--- {'Indian'} ---
Emotion:--- {'Neutral'} ---
------

In [38]:
import os
from os import listdir
folder_dir="/home/manish/Documents/fotos"
for images in os.listdir(folder_dir):
    if(images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
        print(images)
        image1 = Image.open(PATHIMAGE)
        img_tensor= data_transforms(image1)
        imge = Image.open(PATHIMAGE).convert('L')
        img = imge.resize((48,48),Image.Resampling.LANCZOS)
        numpydata = asarray(img)
        res1,res2= democlassResult(img_tensor,numpydata)
        printDemoClassiOutput(res1,res2)

SyntaxError: invalid character in identifier (2236667598.py, line 5)

In [10]:
res1=predict_fer(numpydata,model,True) #democlassi  predict_fer ---> emotion
res2=test_image(img_tensor,resnet_model) #democlassi  test_image ---> age,race,gender

In [12]:
print("Output comparision")
print("---Democlassi--- vs---Deepface---")
print("Age:---",{round(res2[0], 0)},"---vs---",{obj1['age']},"---")
print("Gender:---",{res2[2]},"---vs---",{obj1['gender']},"---")
print("Race:---",{res2[4]},"---vs---",{obj1['dominant_race']},"---")
print("Emotion:---",{res1[1]},"---vs---",{obj1['dominant_emotion']},"---")#res1 for emotion

Output comparision
---Democlassi--- vs---Deepface---
Age:--- {29.0} ---vs--- {22} ---
Gender:--- {'Woman'} ---vs--- {'Man'} ---
Race:--- {'Asian'} ---vs--- {'latino hispanic'} ---
Emotion:--- {'Neutral'} ---vs--- {'fear'} ---


In [13]:
print(res1)

({'Angry': 0.040792275, 'Disgusted': 0.0008434682, 'Afraid': 0.027870916, 'Happy': 0.20536155, 'Sad': 0.08617818, 'Surprised': 0.0066849818, 'Neutral': 0.6322686}, 'Neutral')


In [14]:
print(res2)

(29.182735, {'Man': 0.13909152, 'Woman': 0.8609085}, 'Woman', {'White': 0.15336668, 'Black': 0.22007346, 'Asian': 0.5476374, 'Indian': 0.003787156, 'Unknown-race': 0.07513531}, 'Asian')
